In [1]:
#@title Run to import libraries
!pip install Box2D > /dev/null 2>&1
!pip install gym[box2d] pyvirtualdisplay > /dev/null 2>&1
!pip install plotly --upgrade

In [2]:
!pip install gym[atari]

In [3]:
!pip install tensorboard

In [4]:
!pip install pandas

In [5]:
!pip install pyvirtualdisplay

In [6]:
!pip install torch

In [7]:
!pip install gym[accept-rom-license]

In [8]:
!pip install kornia

In [1]:
%load_ext tensorboard

import math
import gym 
import plotly.express as px
import numpy as np
import warnings
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import random
from collections import Counter, deque
import pandas as pd
from sklearn.model_selection import train_test_split
import datetime
from sklearn.metrics import confusion_matrix
import plotly.figure_factory as ff

import os
import glob
from IPython import display as ipythondisplay
from tqdm.notebook import tqdm
from gym.wrappers import Monitor
from IPython.display import HTML
import base64
import io
import pickle
import torch
from torch import nn
import kornia.augmentation as aug
warnings.filterwarnings("ignore")

/home/alexdloia/miniconda3/lib/python3.7/site-packages/ale_py/roms/utils.py:90: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  for external in metadata.entry_points().get(self.group, []):


In [2]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
# display.start()

In [3]:
import os, sys, copy, argparse, shutil
def parse_arguments():
    parser = argparse.ArgumentParser(description='Deep Q Network Argument Parser')
    parser.add_argument('--seed', dest='seed', type=int, default=1)
    parser.add_argument('--env', dest='env', type=str, default='CartPole-v0')
    parser.add_argument('--save_interval', type=int, default=50, help='save model every n episodes')
    parser.add_argument('--log_interval', type=int, default=10, help='logging every n episodes')
    parser.add_argument('--render', help='render', type=int, default=1)
    parser.add_argument('--batch_size', help='batch_size', type=int, default=32)
    parser.add_argument('--train_freq', help='train_frequency', type=int, default=1)
    parser.add_argument('--max_episode', help='maximum episode', type=int, default=None)
    parser.add_argument('--max_timesteps', help='maximum timestep', type=int, default=100000000)
    parser.add_argument('--lr', dest='lr', type=float, default=0.00025)
    parser.add_argument('--lr_decay', action='store_true', help='decay learning rate')
    parser.add_argument('--gamma', help='discount_factor', type=float, default=0.99)
    parser.add_argument('--warmup_mem', type=int, help='warmup memory size', default=1000)
    parser.add_argument('--frame_skip', type=int, help='number of frames to skip for each action', default=3)
    parser.add_argument('--frame_stack', type=int, help='number of frames to stack', default=4)
    parser.add_argument('--memory', help='memory size', type=int, default=1000000)
    parser.add_argument('--initial_epsilon', '-ie', help='initial_epsilon', type=float, default=0.5)
    parser.add_argument('--final_epsilon', '-fe', help='final_epsilon', type=float, default=0.05)
    parser.add_argument('--max_epsilon_decay_steps', '-eds', help='maximum steps to decay epsilon', type=int, default=100000)
    parser.add_argument('--max_grad_norm', type=float, default=None, help='maximum gradient norm')
    parser.add_argument('--soft_update', '-su', action='store_true', help='soft update target network')
    parser.add_argument('--double_q', '-dq', action='store_true', help='enabling double DQN')
    parser.add_argument('--dueling_net', '-dn', action='store_true', help='enabling dueling network')
    parser.add_argument('--test', action='store_true', help='test the trained model')
    parser.add_argument('--tau', type=float, default=0.01, help='tau for soft target network update')
    parser.add_argument('--hard_update_freq', '-huf', type=int, default=2000, help='hard target network update frequency')
    parser.add_argument('--save_dir', type=str, default='./data')
    parser.add_argument('--resume_step', '-rs', type=int, default=None)
    return parser.parse_args()

In [4]:
#@title Set up constants for env and training
test = False 
save_dir = './data'
render = False
max_episode = None
max_timesteps = 100000000


In [5]:
#@title Augmentations

color_jitter = aug.ColorJitter(
        brightness=np.random.random(),
        contrast=np.random.random(),
        saturation=np.random.random(),
        hue=np.random.random(),
        p=0.5
        )
random_elastic_transform = aug.RandomElasticTransform()
random_fisheye = aug.RandomFisheye(
        center_x=torch.tensor([-.3, .3]),
        center_y=torch.tensor([-.3, .3]),
        gamma=torch.tensor([.9, 1.]),
        )
random_color_equalize = aug.RandomEqualize()
random_gaussian_blur = aug.RandomGaussianBlur(
        kernel_size=(9, 9),
        sigma = (5., 5.)
        )
random_gaussian_noise = aug.RandomGaussianNoise()
random_horizontal_flip = aug.RandomHorizontalFlip()
random_color_invert = aug.RandomInvert()
random_perspective_shift = aug.RandomPerspective()
random_shift = nn.Sequential(aug.RandomCrop((190, 140)), nn.ReplicationPad2d(20), aug.RandomCrop((210, 160)))

In [6]:
def tie_weights(src, trg):
    assert type(src) == type(trg)
    trg.weight = src.weight
    trg.bias = src.bias


# for 84 x 84 inputs
OUT_DIM = {2: 39, 4: 35, 6: 31}
# for 64 x 64 inputs
OUT_DIM_64 = {2: 29, 4: 25, 6: 21}

''' TODO change the layer parameters ''' 
class PixelEncoder(nn.Module):
    """Convolutional encoder of pixels observations."""
    # [210, 160] --> crop [190, 140]
    def __init__(self, obs_shape, feature_dim=50, num_layers=3, num_filters=64, output_logits=False):
        super().__init__()

        assert len(obs_shape) == 3
        self.obs_shape = obs_shape
        self.feature_dim = feature_dim
        self.num_layers = num_layers
        
        # 160, 210, 3 
        self.conv1 = nn.Conv2d(3, 32, kernel_size=5, stride=5)
        # Input to conv2: 32, 42, 32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        # Input to conv3: 15, 20, 64
        self.conv3 = nn.Conv2d(64, 64, kernel_size=4, stride=1)
        # Output from conv3: 12, 17, 64

        # out_dim = OUT_DIM_64[num_layers] if obs_shape[-1] == 64 else OUT_DIM[num_layers]
        out_dims = (12, 17)
        self.fc = nn.Linear(num_filters * out_dims[0] * out_dims[1], self.feature_dim)
        self.ln = nn.LayerNorm(self.feature_dim)

        self.outputs = dict()
        self.output_logits = output_logits

    def reparameterize(self, mu, logstd):
        std = torch.exp(logstd)
        eps = torch.randn_like(std)
        return mu + eps * std

    def forward_conv(self, obs):
        obs = obs / 255.
        self.outputs['obs'] = obs
        conv1 = torch.relu(self.conv1(obs))
        self.outputs['conv1'] = conv1
        conv2 = torch.relu(self.conv2(conv1))
        self.outputs['conv2'] = conv2
        conv3 = torch.relu(self.conv3(conv2))
        self.outputs['conv3'] = conv3

        h = conv3.view(conv3.size(0), -1)
        return h

    def forward(self, obs, detach=False):
        h = self.forward_conv(obs)

        if detach:
            h = h.detach()

        h_fc = self.fc(h)
        self.outputs['fc'] = h_fc

        h_norm = self.ln(h_fc)
        self.outputs['ln'] = h_norm

        if self.output_logits:
            out = h_norm
        else:
            out = torch.tanh(h_norm)
            self.outputs['tanh'] = out

        return out

    def copy_conv_weights_from(self, source):
        """Tie convolutional layers"""
        # only tie conv layers
        for i in range(self.num_layers):
            tie_weights(src=source.convs[i], trg=self.convs[i])

    def log(self, L, step, log_freq):
        if step % log_freq != 0:
            return

        for k, v in self.outputs.items():
            L.log_histogram('train_encoder/%s_hist' % k, v, step)
            if len(v.shape) > 2:
                L.log_image('train_encoder/%s_img' % k, v[0], step)

        for i in range(self.num_layers):
            L.log_param('train_encoder/conv%s' % (i + 1), self.convs[i], step)
        L.log_param('train_encoder/fc', self.fc, step)
        L.log_param('train_encoder/ln', self.ln, step)

In [7]:
def wrap_env(env, train=True):
    suffix = 'train' if train else 'test'
    monitor_dir = os.path.join(save_dir, 'monitor_%s' % suffix)
    os.makedirs(monitor_dir, exist_ok=True)
    if not train:
        video_save_interval = 10
        env = Monitor(env, directory=monitor_dir,
                      video_callable=lambda episode_id: episode_id % video_save_interval == 0,
                      force=True)
    else:
        if render:
            if max_episode is not None:
                video_save_interval = int(max_episode / 3)
            else:
                video_save_interval = int(max_timesteps / float(env._max_episode_steps) / 3)
            env = Monitor(env, directory=monitor_dir,
                          video_callable=lambda episode_id: episode_id % video_save_interval == 0,
                          force=True)
        else:
            env = Monitor(env, directory=monitor_dir, video_callable=False, force=True)
    return env

In [8]:
class ReplayMemory(object):
    def __init__(self, max_epi_num=2000, max_epi_len=200):
        # capacity is the maximum number of steps in memory
        self.max_epi_num = max_epi_num
        self.max_epi_len = max_epi_len
        # saves each tuple of (state, action, next state, reward)
        self.capacity = 1000 # self.max_epi_num * max_epi_len
        self.idx = 0
        self.obs_memory = np.zeros((self.capacity, 210, 160, 3)) # deque(maxlen=self.max_epi_num * max_epi_len)
        self.next_memory = np.zeros((self.capacity, 210, 160, 3))
        self.act_memory = np.zeros((self.capacity, 1))
        self.reward_memory = np.zeros((self.capacity, 1))
        self.is_av = False
        self.current_epi = 0

    def reset(self):
        self.current_epi = 0
        self.memory.clear()

    ''' deprecated for tuple buffer '''
    def create_new_epi(self):
        pass

    def remember(self, state, next_state, action, reward):
        if self.idx == self.capacity:
            self.idx = 0
        self.obs_memory[self.idx] = state.copy()
        self.next_memory[self.idx] = next_state.copy()
        self.act_memory[self.idx] = action
        self.reward_memory[self.idx] = reward
        self.idx += 1
        
        '''
        if len(self.memory) < self.capacity:
            new_sample = np.array([state, action, reward, next_state])
            if len(self.memory) == 0:
                self.memory = [new_sample]
            else:
                length = len(self.memory)
                self.memory.append(new_sample)
        '''
                
    # samples batch_size
    def sample(self, batch_size):
        if batch_size < self.idx:
            idx = np.random.randint(0, self.idx - 1, batch_size)
            return self.obs_memory[idx], self.next_memory[idx], self.act_memory[idx], self.reward_memory[idx]
        return self.obs_memory, self.next_memory, self.act_memory, self.reward_memory

    def size(self):
        return self.idx

    def is_available(self):
        self.is_av = True
        if self.idx <= 1:
            self.is_av = False
        return self.is_av

    def print_info(self):
        pass

In [9]:
#@title Create a training conv agent
import torch.nn.functional as F

class DQNetworkConv(nn.Module):
    def __init__(self, in_channels, act_dim, dueling=False):
        super(DQNetworkConv, self).__init__()
        self.act_dim = act_dim
        self.dueling = dueling
        # 160, 210, 3 
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=5, stride=5)
        # Input to conv2: 32, 42, 32
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        # Input to conv3: 15, 20, 64
        self.conv3 = nn.Conv2d(64, 64, kernel_size=4, stride=1)
        # Output from conv3: 12, 17, 64
        if self.dueling:
            self.v_fc4 = nn.Linear(12 * 17 * 64, 512)
            self.adv_fc4 = nn.Linear(12 * 17 * 64, 512)
            self.v_fc5 = nn.Linear(512, 1)
            self.adv_fc5 = nn.Linear(512, self.act_dim)
        else:
            self.fc4 = nn.Linear(12 * 17 * 64, 512)
            self.fc5 = nn.Linear(512, self.act_dim)
        self.parameters = (list(self.conv1.parameters())) + (list(self.conv2.parameters())) + (list(self.conv3.parameters())) + (list(self.fc4.parameters())) + (list(self.fc5.parameters()))

    def forward(self, st):
        out = F.relu(self.conv1(st))
        out = F.relu(self.conv2(out))
        out = F.relu(self.conv3(out))
        out = out.view(out.size(0), -1)
        if self.dueling:
            val = F.relu(self.v_fc4(out))
            adv = F.relu(self.adv_fc4(out))
            val = self.v_fc5(val)
            adv = self.adv_fc5(adv)
            out = val.expand_as(adv) + adv - adv.mean(-1, keepdim=True).expand_as(adv)
        else:
            out = F.relu(self.fc4(out))
            out = self.fc5(out)
        return out

In [10]:
#@title Create a training FC agent
import torch.nn.functional as F

class DQNetworkFC(nn.Module):
    def __init__(self, z_dim, act_dim, dueling=False):
        super(DQNetworkFC, self).__init__()
        self.act_dim = act_dim
        self.input_dim = z_dim 
        self.dueling = dueling
        if self.dueling:
            self.v_fc1 = nn.Linear(z_dim, 512)
            self.adv_fc1 = nn.Linear(z_dim, 512)
            self.v_fc2 = nn.Linear(512, 1)
            self.adv_fc2 = nn.Linear(512, 256)
            self.v_fc3 = nn.Linear(256, 1)
            self.adv_fc3 = nn.Linear(256, self.act_dim)
        else:
            self.fc1 = nn.Linear(z_dim, 512)
            self.fc2 = nn.Linear(512, 256)
            self.fc3 = nn.Linear(256, self.act_dim)

    def forward(self, st):
        out = F.relu(self.fc1(st))
        out = F.relu(self.fc2(out))
        ''' Do we need a relu on the last layer if the output is probability over action space? '''
        out = F.relu(self.fc3(out))
        return out

In [11]:
def process_obs(obs):
    obs = torch.Tensor(obs)
    if len(obs.shape) < 4:
        obs = obs.unsqueeze(0)
    obs = obs.permute(0, 3, 1, 2)
    return obs

In [12]:
def take_action(env, action):
    state, rew, done, _ = env.step(action)
    obs = env.render(mode='rgb_array')
    return obs, rew, done

In [13]:
MAX_STEPS = 200

In [14]:
class CURL(nn.Module):
    """
    CURL
    """

    def __init__(self, obs_shape, z_dim, batch_size, encoder, output_type="continuous", critic=None, critic_target=None):
        super(CURL, self).__init__()
        self.batch_size = batch_size

        # self.encoder = critic.encoder
        self.encoder = encoder 

        # self.encoder_target = critic_target.encoder 

        self.W = nn.Parameter(torch.rand(z_dim, z_dim))
        self.output_type = output_type

    def encode(self, x, detach=False, ema=False):
        """
        Encoder: z_t = e(x_t)
        :param x: x_t, x y coordinates
        :return: z_t, value in r2
        """
        if ema:
            with torch.no_grad():
                z_out = self.encoder_target(x)
        else:
            z_out = self.encoder(x)

        if detach:
            z_out = z_out.detach()
        return z_out

    def compute_logits(self, z_a, z_mod):
        """
        Uses logits trick for CURL:
        - compute (B,B) matrix z_a (W z_pos.T)
        - positives are all diagonal elements
        - negatives are all other elements
        - to compute loss use multiclass cross entropy with identity matrix for labels
        """
        Wz = torch.matmul(self.W, z_mod.T)  # (z_dim,B)
        logits = torch.matmul(z_a, Wz)  # (B,B)
        logits = logits - torch.max(logits, 1)[0][:, None]
        return logits

In [15]:
#@title Generate a batch of negatively labelled examples given observations

def generate_negatives(obs):
    neg_idx = np.random.randint(len(obs), size=len(obs))
    pos_idx = np.arange(len(obs))
    resample = (neg_idx == pos_idx)
    for (i, r) in enumerate(resample):
        if r:
            idx = neg_idx[i]
        else:
            idx = np.random.randint(0, len(obs), 1)[0]
            while idx == i:
                idx = np.random.randint(0, len(obs), 1)[0]
        neg_idx[i] = idx
    return (obs[neg_idx]).copy()


In [16]:
#@title Create a training agent (wrapper for conv agent)

GAMMA = 0.99

class Agent(object):
    def __init__(self, act_dim, in_channels=3, max_epi_num=50, max_epi_len=300, CURL=None, conv_net=False):
        self.N_action = act_dim
        self.max_epi_num = max_epi_num
        self.max_epi_len = max_epi_len
        ''' To decide when to copy weights to the target network '''
        self.num_param_updates = 0
        self.CURL = CURL
        if conv_net:
            self.conv_net = DQNetworkConv(in_channels, act_dim)
            self.target = DQNetworkConv(in_channels, act_dim)
        else:
            ''' if using the encoder head for contrastive loss '''
            self.conv_net = DQNetworkFC(self.CURL.encoder.feature_dim, act_dim)
            self.target = DQNetworkFC(self.CURL.encoder.feature_dim, act_dim)
        self.buffer = ReplayMemory(max_epi_num=self.max_epi_num, max_epi_len=self.max_epi_len)
        self.gamma = 0.999
        self.loss_fn = torch.nn.MSELoss()
        self.optimizer = torch.optim.Adam(list(self.conv_net.parameters()) + list(self.CURL.parameters()) + list(self.CURL.encoder.parameters()), lr=1e-3)

    def remember(self, state, action, reward, next_state):
        self.buffer.remember(state, next_state, action, reward)

    ''' Copy the weights to the target network every 100 updates '''
    def train(self, batch_size=32, target_update_freq=100, use_encoder=True):
        if self.buffer.is_available():
            obs, next_obs, action_list, reward_list = self.buffer.sample(batch_size)
            
            ''' Pass through the encoder to get encodings
             If also training the contrastive loss
             include that here! 
             1. data augmentation to create pos and negative pairs
             2. encoder 
             3. update encoder loss function (using a separate optimizer) or add to the loss computed below
            '''

            losses = [] 

            # check if obs is a numpy or a torch tensor
            if use_encoder:
                obs_anchor = process_obs(obs.copy())
                obs_pos = random_shift(obs_anchor)
                # mixed_obs = generate_negatives(obs)
                # mixed_obs = process_obs(mixed_obs)
                # obs_neg = random_shift(mixed_obs)
                z_a = self.CURL.encode(obs_anchor)
                z_pos = self.CURL.encode(obs_pos)
                # Mix pairs to generate negative labels
                # z_neg = self.CURL.encode(obs_neg)
                next_obs = process_obs(next_obs.copy())
                z_next = self.CURL.encode(next_obs)

                logits = self.CURL.compute_logits(z_a, z_pos)
                labels = torch.arange(logits.shape[0]).long()
                '''
                pos_logits = self.CURL.compute_logits(z_a, z_pos)
                neg_logits = self.CURL.compute_logits(z_a, z_neg)
                # [32, 32]
                pos_labels = torch.ones(pos_logits.shape[0]).long()
                neg_labels = torch.zeros(neg_logits.shape[0]).long() 
                # TODO: stack pos and neg logits and labels (double check dim)
                logits = torch.stack([pos_logits, neg_logits]).squeeze(0)
                labels = torch.stack([pos_labels, neg_labels]).squeeze(0)
                '''
                
                # pass into the loss function
                encoding_loss = nn.CrossEntropyLoss()(logits, labels)

                ''' Combine encoding loss with rl loss below '''
                losses.append(encoding_loss)

                # Then pass that encoding through the conv_net to get Q value estimates
                Qs = self.conv_net(z_a)
                next_Qs = self.target(z_next).detach().max(1)[0]
            
            else:
                ''' If not using the encoder, pass the obs directly to the CNN '''
                obs = process_obs(obs)
                # estimate current q values from observations
                Qs = self.conv_net(obs)
                # find next max q values based on next observations
                next_Qs = self.target(next_obs).detach().max(1)[0]
            
            ''' find target q values ''' 
            next_Qs = next_Qs.numpy()
            Qs = torch.gather(Qs, dim=1, index=torch.tensor(action_list, dtype=torch.int64))
            target_Qs = torch.tensor(reward_list.squeeze(-1) + GAMMA * next_Qs).long()
            ''' try to set Qs equal to target_Qs '''
            q_loss = self.loss_fn(Qs, target_Qs).long()
            losses.append(q_loss)
            
            ''' Loss update for q network and encoder head '''
            losses = torch.stack(losses).sum()
            self.optimizer.zero_grad()
            losses.backward()
            self.optimizer.step()

            self.num_param_updates += 1
            if self.num_param_updates % target_update_freq == 0:
                self.target.load_state_dict(self.conv_net.state_dict())

    # TODO: check the sizes of inputs and outputs
    def get_action(self, obs, epsilon, use_encoding=True):
        ''' 
         If using an encoder, need to pass that thorugh the encoder
         then use the encoding to pass through self.conv_net
        '''
        # obs = torch.tensor(obs)
        if use_encoding:
            obs = process_obs(obs)
            obs = self.CURL.encode(obs).detach()

        # Dividing obs by 255 is handled in encoder forward (only needed for use_encoding=False)
        if len(obs.shape) == 1:
            obs = obs.unsqueeze(0)

        # epsilon greedy for selecting which action to take
        if random.random() > epsilon:
            qs = self.conv_net(obs)
            action = qs[0].argmax().data.item()
        else:
            action = random.randint(0, self.N_action-1)

        return action

def get_decay(epi_iter):
    decay = math.pow(0.999, epi_iter)
    if decay < 0.05:
        decay = 0.05
    return decay

In [17]:
def main():
    env = gym.make('ALE/SpaceInvaders-v5')
    max_epi_iter = 2000
    max_MC_iter = 200
    obs = env.render(mode='rgb_array')
    encoder = PixelEncoder(obs_shape=obs.shape, feature_dim=50, num_layers=2, num_filters=64, output_logits=False)
    CURL_encoder = CURL(obs_shape=obs.shape, z_dim=50, batch_size=1, encoder=encoder, output_type="continuous")
    agent = Agent(act_dim=env.action_space.n, max_epi_num=1000, max_epi_len=max_MC_iter, CURL=CURL_encoder)
    train_curve = []
    for epi_iter in range(max_epi_iter):
        random.seed()
        env.reset()
        obs = env.render(mode='rgb_array')
        returns = 0.0
        for MC_iter in range(max_MC_iter):
            # Make sure to set this to an actual action
            # TODO: decide whether to normalize pixels by / 255 (already done in PixelEncoder atm)
            action = agent.get_action(obs, get_decay(epi_iter))
            next_obs, reward, done = take_action(env, action)
            returns += reward * agent.gamma ** (MC_iter)
            agent.remember(obs, action, reward, next_obs)
            obs = next_obs.copy()
            if done or MC_iter >= max_MC_iter-1:
                agent.buffer.create_new_epi()
                break
        print('Episode', epi_iter, 'returns', returns)
        if epi_iter % 1 == 0:
            train_curve.append(returns)
        if agent.buffer.is_available():
            agent.train()
        
    print(train_curve)

In [18]:
main()

A.L.E: Arcade Learning Environment (version +978d2ce)
[Powered by Stella]


Episode 0 returns 26.077032731657333
Episode 1 returns 35.91208460741555
Episode 2 returns 51.671238414286876
Episode 3 returns 38.00912762762846
Episode 4 returns 39.19130008478749
Episode 5 returns 44.69249570464702
Episode 6 returns 53.721795518938364
Episode 7 returns 0.0
Episode 8 returns 91.01033348774561
Episode 9 returns 29.821626325799013
Episode 10 returns 81.30449668051236
Episode 11 returns 57.57098608783476
Episode 12 returns 57.01334526101368
Episode 13 returns 26.07110407825337
Episode 14 returns 41.03784433426074
Episode 15 returns 80.43846310222212
Episode 16 returns 27.29877528285314
Episode 17 returns 40.05740070644874
Episode 18 returns 67.29723987185868
Episode 19 returns 66.1483267440132
Episode 20 returns 53.287376585976375
Episode 21 returns 38.495658069829915
Episode 22 returns 12.596350460750356
Episode 23 returns 42.881824917676546
Episode 24 returns 39.699308431196165
Episode 25 returns 18.252839113237265
Episode 26 returns 8.942356600819899
Episode 27 retur

Episode 218 returns 78.65599960689107
Episode 219 returns 58.752269170840634
Episode 220 returns 49.64610023019273
Episode 221 returns 29.34198986325618
Episode 222 returns 8.829761711339113
Episode 223 returns 26.367490372395288
Episode 224 returns 27.08069437951174
Episode 225 returns 32.799597943841135
Episode 226 returns 74.29313607327207
Episode 227 returns 71.42330157345191
Episode 228 returns 77.09599864744294
Episode 229 returns 35.26739822961046
Episode 230 returns 30.65174967413028
Episode 231 returns 30.71188344891183
Episode 232 returns 63.59170339699436
Episode 233 returns 81.28606426841463
Episode 234 returns 60.82793487766425
Episode 235 returns 67.75226419097731
Episode 236 returns 72.50779344162397
Episode 237 returns 26.50484462641507
Episode 238 returns 52.87474305636254
Episode 239 returns 76.22553526677646
Episode 240 returns 61.40126292663049
Episode 241 returns 64.50429495170872
Episode 242 returns 78.77879871144334
Episode 243 returns 97.54898263891499
Episode 2

Episode 432 returns 125.3725496383542
Episode 433 returns 21.88019119078242
Episode 434 returns 51.13958121794981
Episode 435 returns 142.96636376842199
Episode 436 returns 4.294614782805264
Episode 437 returns 37.08730818614036
Episode 438 returns 91.53458930764864
Episode 439 returns 26.229146763649737
Episode 440 returns 52.57761617549064
Episode 441 returns 52.082322115957055
Episode 442 returns 44.76866484188496
Episode 443 returns 26.592993796603192
Episode 444 returns 55.56149982535024
Episode 445 returns 53.628616881426986
Episode 446 returns 44.41525307890666
Episode 447 returns 21.650494949723353
Episode 448 returns 83.98943837472208
Episode 449 returns 56.990948429051905
Episode 450 returns 29.860698733131663
Episode 451 returns 106.83924481508555
Episode 452 returns 21.519920697809795
Episode 453 returns 41.62172128557688
Episode 454 returns 50.188982051474866
Episode 455 returns 4.256117495685137
Episode 456 returns 38.23480161463506
Episode 457 returns 86.13001982170374
E

Episode 647 returns 18.212281017525704
Episode 648 returns 23.143528164041975
Episode 649 returns 67.98549583813688
Episode 650 returns 31.39929515843867
Episode 651 returns 55.84825231138336
Episode 652 returns 17.5706288855806
Episode 653 returns 9.060644977231139
Episode 654 returns 21.31649714155488
Episode 655 returns 36.87783064332881
Episode 656 returns 30.02383184055687
Episode 657 returns 8.688809576254986
Episode 658 returns 17.573234292383738
Episode 659 returns 77.28525730138617
Episode 660 returns 54.044369295077715
Episode 661 returns 47.64575520478914
Episode 662 returns 71.28345359354044
Episode 663 returns 16.823035446111927
Episode 664 returns 75.88737333835502
Episode 665 returns 4.452117713883588
Episode 666 returns 31.084653386446455
Episode 667 returns 80.11819590745584
Episode 668 returns 26.04858597291736
Episode 669 returns 67.44826295556594
Episode 670 returns 64.2114217346155
Episode 671 returns 63.27389166295405
Episode 672 returns 12.5404653202053
Episode 6

...

Episode 1076 returns 81.05166625280353
Episode 1077 returns 53.40572224632181
Episode 1078 returns 17.844263761923724
Episode 1079 returns 48.594411964432545
Episode 1080 returns 46.40448204426359
Episode 1081 returns 35.121952571588984
Episode 1082 returns 45.513623172622005
Episode 1083 returns 71.01261811966273
Episode 1084 returns 89.83754301557417
Episode 1085 returns 99.19402579643757
Episode 1086 returns 27.034288605829374
Episode 1087 returns 84.49877030956382
Episode 1088 returns 8.460258619615672
Episode 1089 returns 49.589641802855105
Episode 1090 returns 71.0151591137056
Episode 1091 returns 36.468899444803675
Episode 1092 returns 30.500621008185412
Episode 1093 returns 54.079148505677075
Episode 1094 returns 117.20560508795499
Episode 1095 returns 36.21963956714609
Episode 1096 returns 41.64851746443418
Episode 1097 returns 50.04769911479421
Episode 1098 returns 13.149504946694709
Episode 1099 returns 45.76837858818821
Episode 1100 returns 55.03835194193037
Episode 1101 re

Episode 1286 returns 13.747785424774104
Episode 1287 returns 31.79063181005207
Episode 1288 returns 17.817538317609092
Episode 1289 returns 31.9962209746719
Episode 1290 returns 13.426344040075419
Episode 1291 returns 46.289225988211285
Episode 1292 returns 86.69997814779035
Episode 1293 returns 76.4193031737356
Episode 1294 returns 54.2647219992096
Episode 1295 returns 59.18992936467308
Episode 1296 returns 35.73538182676516
Episode 1297 returns 40.86409511662015
Episode 1298 returns 31.584173036822705
Episode 1299 returns 36.2525545486189
Episode 1300 returns 21.87171138606034
Episode 1301 returns 26.08469741857349
Episode 1302 returns 44.486721838222984
Episode 1303 returns 116.90579419025937
Episode 1304 returns 46.33168580124911
Episode 1305 returns 58.78787507532001
Episode 1306 returns 94.32610938265091
Episode 1307 returns 40.60299062285924
Episode 1308 returns 31.210963760062256
Episode 1309 returns 55.36878546981882
Episode 1310 returns 31.4188892357027
Episode 1311 returns 6

Episode 1495 returns 37.38701500963066
Episode 1496 returns 46.077912598253185
Episode 1497 returns 49.7412511461361
Episode 1498 returns 31.66866527989874
Episode 1499 returns 36.149234888549316
Episode 1500 returns 84.82357935879456
Episode 1501 returns 35.90217941172225
Episode 1502 returns 30.835069870203828
Episode 1503 returns 35.70892183242786
Episode 1504 returns 48.336984902310576
Episode 1505 returns 93.51296441012946
Episode 1506 returns 4.324797697150319
Episode 1507 returns 54.31367510708606
Episode 1508 returns 23.406259955236532
Episode 1509 returns 54.254923031144344
Episode 1510 returns 26.69207838486903
Episode 1511 returns 72.02741487805399
Episode 1512 returns 21.026620276245964
Episode 1513 returns 13.149335604716692
Episode 1514 returns 54.50525487624911
Episode 1515 returns 31.40317133181988
Episode 1516 returns 49.32135812951696
Episode 1517 returns 53.21331428027807
Episode 1518 returns 49.20643178294846
Episode 1519 returns 98.95104347712866
Episode 1520 retur

Episode 1705 returns 4.5239607355685445
Episode 1706 returns 22.595532850286535
Episode 1707 returns 61.54494091649926
Episode 1708 returns 44.13811977055338
Episode 1709 returns 31.00037450193428
Episode 1710 returns 76.47591945262431
Episode 1711 returns 27.103984460550425
Episode 1712 returns 54.64171579095138
Episode 1713 returns 18.246033285958216
Episode 1714 returns 17.92408177165628
Episode 1715 returns 58.338736808523834
Episode 1716 returns 26.687773487072974
Episode 1717 returns 35.55236853315783
Episode 1718 returns 47.64875372103996
Episode 1719 returns 13.558799894292482
Episode 1720 returns 48.31052058464855
Episode 1721 returns 74.80022856137211
Episode 1722 returns 8.426021379747336
Episode 1723 returns 31.0112512088658
Episode 1724 returns 0.0
Episode 1725 returns 26.977831495342393
Episode 1726 returns 36.40007494496027
Episode 1727 returns 40.970889054702
Episode 1728 returns 12.936624203651888
Episode 1729 returns 35.92709256109653
Episode 1730 returns 26.463096834

Episode 1915 returns 29.396127680554798
Episode 1916 returns 45.82600762461999
Episode 1917 returns 31.719450165674356
Episode 1918 returns 22.681669164480553
Episode 1919 returns 36.27073482821314
Episode 1920 returns 8.775044936794856
Episode 1921 returns 27.628717182813887
Episode 1922 returns 40.958566048981346
Episode 1923 returns 35.988562099154855
Episode 1924 returns 4.56488070912048
Episode 1925 returns 9.283104953747497
Episode 1926 returns 26.413686347829
Episode 1927 returns 44.740614348244705
Episode 1928 returns 38.750104725782585
Episode 1929 returns 22.680200138442896
Episode 1930 returns 9.069037586951147
Episode 1931 returns 38.6535033626091
Episode 1932 returns 45.70273605431255
Episode 1933 returns 33.07504306620039
Episode 1934 returns 8.474235482751466
Episode 1935 returns 4.79904720262538
Episode 1936 returns 86.80558400185393
Episode 1937 returns 40.67662818840948
Episode 1938 returns 76.397276839993
Episode 1939 returns 40.045206134593286
Episode 1940 returns 5

In [ ]:
def old_main():
    env = gym.make('MountainCars-v0')
    if len(env.observation_space.shape) >= 3:
        env = WrapAtariEnv(env=env, noop_max=30, frameskip=3, framestack=4, test=test)
    if not test:
        dele = input("Do you wanna recreate ckpt and log folders? (y/n)")
        if dele == 'y':
            if os.path.exists(save_dir):
                shutil.rmtree(save_dir)

    env = wrap_env(env, train=not test)
    print(env.observation_space.shape)
    if len(env.observation_space.shape) >= 3:
        q_net = DQNetworkConv
    else:
        assert(False)
    agent = DQNAgent(env=env, qnet=q_net)
    if args.test:
        agent.rollout(episodes=100, render=render)
    else:
        agent.train()
    agent.env.close()

In [ ]:
def main_mountaincar():
    env = gym.make('ALE/SpaceInvaders-v5')
    env.reset()

    for i in range(5):
        env.step(env.action_space.sample())
        obs = env.render(mode='rgb_array')
        print("Step ", i, obs)
    env.close()

In [ ]:
main_mountaincar()

Code references for DQN:

https://github.com/taochenshh/dqn-pytorch

https://github.com/transedward/pytorch-dqn (for sampling from replay buffer)

CURL code: https://github.com/MishaLaskin/curl